In [1]:
import pandas as pd
import numpy as np

In [2]:
customer = pd.read_excel("customer_details.xlsx")

In [3]:
customer.head()

,Customer Id,Customer mail
0,15889,customer15889@gmail.com
1,15890,customer15890@gmail.com
2,15892,customer15892@gmail.com
3,15893,customer15893@gmail.com
4,15894,customer15894@gmail.com


In [4]:
products = pd.read_excel("products.xlsx")
products.head()

,products
0,Medium-term deposits
1,Securities
2,Direct Debit
3,Payroll Account
4,Pensions


In [5]:
rate = pd.read_csv("ratings.csv",usecols = ['rating'])

In [6]:
rate.head()

,rating
0,4.0
1,4.0
2,4.0
3,5.0
4,5.0


In [7]:
rating = rate['rating'].unique()

In [8]:
rating

array([4. , 5. , 3. , 2. , 1. , 4.5, 3.5, 2.5, 0.5, 1.5])

In [9]:
rate = []
for i in range(len(customer)):
    i = np.random.choice(rating)
    rate.append(i)

customer['cust_rating'] = rate

In [10]:
customer.head()

,Customer Id,Customer mail,cust_rating
0,15889,customer15889@gmail.com,2.0
1,15890,customer15890@gmail.com,2.5
2,15892,customer15892@gmail.com,5.0
3,15893,customer15893@gmail.com,2.0
4,15894,customer15894@gmail.com,1.5


In [11]:
prod = []
for i in range(len(customer)):
    i = np.random.choice(products['products'].to_list())
    prod.append(i)
customer['Cust_Products'] = prod 

In [12]:
customer.head()

,Customer Id,Customer mail,cust_rating,Cust_Products
0,15889,customer15889@gmail.com,2.0,Medium-term deposits
1,15890,customer15890@gmail.com,2.5,Payroll
2,15892,customer15892@gmail.com,5.0,Current Accounts
3,15893,customer15893@gmail.com,2.0,Direct Debit
4,15894,customer15894@gmail.com,1.5,e-account


In [16]:
feature = customer.pivot_table(index = 'Cust_Products',columns = 'Customer Id',values = 'cust_rating').fillna(0)
feature.head()

Customer Id,15889,15890,15892,15893,15894,15895,15896,15897,15898,15899,...,1553679,1553680,1553681,1553683,1553684,1553685,1553686,1553687,1553688,1553689
Cust_Products,,,,,,,,,,,,,,,,,,,,,
Credit Card,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Current Accounts,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Derivada Account,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Direct Debit,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Funds,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#here we convert this pivot matrix into array matrix
from scipy.sparse import csr_matrix
mat = csr_matrix(feature.values)

In [18]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model.fit(mat)

NearestNeighbors(algorithm='brute', metric='cosine')

In [20]:
mat.shape

(23, 929615)

In [21]:
test = np.random.choice(mat.shape[0])

In [22]:
print(test)

16


In [24]:
distance, indices =model.kneighbors(feature.iloc[test,:].values.reshape(1,-1),n_neighbors = 6) 

In [31]:
for i in range(0,len(distance.flatten())):
    if i == 0:
        print('Rcommendation for {0}:\n'.format(feature.index[test]))
    else:
        print('{0}:{1}, with distance of {2}: '.format(i,feature.index[indices.flatten()[i]],distance.flatten()[i]))

Rcommendation for Saving Account:

1:Securities, with distance of 1.0: 
2:Short-term deposits, with distance of 1.0: 
3:Payroll Account, with distance of 1.0: 
4:Pensions, with distance of 1.0: 
5:Mortgage, with distance of 1.0: 


In [32]:
customer.to_csv('bank_product_recom.csv')